# Scaled Dot-Product Attention

**Self-Attention** is a mechanism that captures dependencies and relationships within input sequences.

**What it does** — for each token, self-attention builds a **contextualized vector** by mixing information from **all** tokens in the sequence, weighted by how relevant they are to that token.
- It allows the model to identify and weight the importance of different parts of the input sequence by attending to itself.

<img src="../images/scaled-dot-product-attn.png" alt="scaled-dot-product-attn" width="400"/>

#### Attention equation:
$$
\operatorname{Attention}(\textbf{Q}, \textbf{K}, \textbf{V})=\underbrace{\operatorname{softmax}\left(\frac{\textbf{Q}\textbf{K}^{\top}}{\sqrt{d_k}}+\operatorname{mask}\right)}_{\text {attention weights } A} \textbf{V}
$$
- $Q \in \mathbb{R}^{n \times d_{k}}, K \in \mathbb{R}^{n \times d_{k}}, V \in \mathbb{R}^{n \times d_{v}}$, with sequence length $ n$.
- $A \in \mathbb{R}^{n \times n}$ has **row-wise** softmax: each row sums to 1.

**Why the scale $\sqrt{d_{k}}$ ?**

Dot products grow with dimension; dividing by $\sqrt{ d_{k} }$ keeps logits in a range where softmax is well-behaved (prevents tiny gradients / saturation).

#### Where $Q, K, V$ come from?
$Q, K, V$ are **learned [linear projections](https://en.wikipedia.org/wiki/Projection_(linear_algebra))** (not literal copies) of the same input $H \in \mathbb{R}^{n \times d}$:

$$
Q=HW_{Q},\ \ K=HW_{K},\ \ V=HW_{V}
$$

This lets the model learn:
- **Q (Query):** what to ask.
- **K (Key):** how to index.
- **V (Value):** what to retrieve.

**Hash-table intuition (soft lookup)**
- Compare each **query** to all **keys** via dot product –> *similarity scores*.
- Softmax turns scores into a probability distribution –> *attention weights*.
- Take a weighted sum of **values** using those weights –> the output for that token.

#### Masks (what to zero out and why)
- **Padding mask:** add $-\infty$ to logits where positions are padding so their softmax weight becomes 0.
- **Causal mask (decoder):** forbid looking ahead (future tokens).
> The mask is added **before** softmax, it zeroes weights *after* softmax.

#### What the $Q, K, V$ matrices mean
- $QK^\top$ –> an $ n \times n$ **score matrix** where entry $(i, j)$ is how much token $i$ attends to token $j$.
- After softmax, each row is a distribution over **which positions token $i$ should read from.**
- Multiplying by $V$ aggregates information: token $i$ gets a weighted mix of other token’s value vectors.
> Important: rows are **not** forced to be one-hot. They’re usually **soft** (spread across many tokens), especially in early layers.

In [1]:
import torch
import torch.nn.functional as F

# 3 tokens, head_dim = 2 tensors
Q = torch.tensor([[1.0, 0.0], [0.0, 1.0], [1.0, 1.0]])
K = torch.tensor([[1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])
V = torch.tensor([[1.0, 0.0], [0.0, 2.0], [1.0, 1.0]])
L, d_h = Q.shape  # L=3, d_h=2

# attention scores
scores = (Q @ K.T) / (d_h**0.5)
print(f"scores shape: {scores.shape}")
print(scores)

# attention weights: softmax over keys dimension
attn_weights = F.softmax(scores, dim=-1)  # [L, L]
print(f"\nattn_weights shape: {attn_weights.shape}")
print(attn_weights)

# output: [L, d_h]
attn_output = attn_weights @ V
print(f"\nattn_output shape: {attn_output.shape}")
print(attn_output)

scores shape: torch.Size([3, 3])
tensor([[0.7071, 0.7071, 0.0000],
        [0.0000, 0.7071, 0.7071],
        [0.7071, 1.4142, 0.7071]])

attn_weights shape: torch.Size([3, 3])
tensor([[0.4011, 0.4011, 0.1978],
        [0.1978, 0.4011, 0.4011],
        [0.2483, 0.5035, 0.2483]])

attn_output shape: torch.Size([3, 2])
tensor([[0.5989, 1.0000],
        [0.5989, 1.2033],
        [0.4965, 1.2552]])
